In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast

In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'Length'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

strategy_version_input=702
attribute_id_input=2425

In [3]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]
# buckets

In [5]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
    
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
    
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    
#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))

from fractions import Fraction
def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))
# measure='This is a fraction: 100 1/64'
# [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A



In [6]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
df = df[df['bucket_id'].isin(buckets)]
len(df)

6645

In [7]:
def regex_patterns(unit):
    return fr'(?i)(\d+\.\d+[\W]?[\W]?[\W]{unit}\b)|(\d+.?\d+\/\d+[\W]?[\W]?[\W]{unit}\b)|((?<!\.)(?<!\/)\d+(?!\.\d)[\W]?[\W]?[\W]{unit}\b)|()'
print(regex_patterns('Le?n?g?t?h?'))
print(regex_patterns('He?i?g?h?t?'))
print(regex_patterns('Wi?d?t?h?'))
print(regex_patterns('Diam?e?t?e?r?'))

(?i)(\d+\.\d+[\W]?[\W]?[\W]Le?n?g?t?h?\b)|(\d+.?\d+\/\d+[\W]?[\W]?[\W]Le?n?g?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.\d)[\W]?[\W]?[\W]Le?n?g?t?h?\b)|()
(?i)(\d+\.\d+[\W]?[\W]?[\W]He?i?g?h?t?\b)|(\d+.?\d+\/\d+[\W]?[\W]?[\W]He?i?g?h?t?\b)|((?<!\.)(?<!\/)\d+(?!\.\d)[\W]?[\W]?[\W]He?i?g?h?t?\b)|()
(?i)(\d+\.\d+[\W]?[\W]?[\W]Wi?d?t?h?\b)|(\d+.?\d+\/\d+[\W]?[\W]?[\W]Wi?d?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.\d)[\W]?[\W]?[\W]Wi?d?t?h?\b)|()
(?i)(\d+\.\d+[\W]?[\W]?[\W]Diam?e?t?e?r?\b)|(\d+.?\d+\/\d+[\W]?[\W]?[\W]Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.\d)[\W]?[\W]?[\W]Diam?e?t?e?r?\b)|()


# NA

In [8]:
regex_pattern_length=r'(?i)(\d+\.\d+.?.?.?Le?n?g?t?h?\b)|(\d+.?\d+\/\d+.?.?.?Le?n?g?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Le?n?g?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Le?n?g?t?h?\b)|()'         
df['matches_length'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
regex_pattern_height=r'(?i)(\d+\.\d+[\W]?[\W]?[\W]?He?i?g?h?t?\b)|(\d+.?\d+\/\d+[\W]?[\W]?[\W]?He?i?g?h?t?\b)|((?<!\.)(?<!up.to.)(?<![\w])(?<!\/)(?<!\/)\d+(?!\.\d)(?<!\d\\?\"?\-\d\d)[\W]?[\W]?[\W]?He?i?g?h?t?(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)\b)|()'        
df['matches_height'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
regex_pattern_width=r'(?i)(\d+\.\d+.?.?.?Wi?d?t?h?\b)|(\d+.?\d+\/\d+.?.?.?Wi?d?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Wi?d?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Wi?d?t?h?\b)|()'
df['matches_width'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_width, x))
regex_pattern_diameter=r'(?i)(\d+\.\d+.?.?.?Diam?e?t?e?r?\b)|(\d+.?\d+\/\d+.?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Diam?e?t?e?r?\b)|()'
df['matches_diameter'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))
regex_pattern_ft=r"\d'.?.?.?[LWHD]"
df['matches_ft'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
regex_pattern_blank=r'\d'
df['new_matches'] = df['matches_length'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
regex_pattern_blank=r'(?i)(\d+.?\d*\/?\d*.?I?n?c?h?"?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?)|(\d+.?ft)|(\d+.?foot)|(\d+.?feet)|(\d+.?inch)|(\d+.?")|(\d+")|()'
df['blank_matches'] = df['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df['blank_matches_long'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

In [9]:
df[df['blank_matches'].astype(str)!='[]'][0:500]

product_id external_id  \
0      1374560670    43383689   
1      1359946081    69758729   
2      1359946053    69758713   
3      1359945976    69758752   
4      1359945913    69758955   
5      1359945909    69758691   
6      1359945899    69758702   
13     1359946130    69758697   
14     1377456243    69591739   
23      994667760    68566968   
24      817612246    68567705   
25      817612132    68567255   
26      817337241    66539056   
27         476947    43346288   
28     1359946070    69758742   
31     1336514615    62786744   
37     1229080824    62289702   
44     1337753379    62138741   
46     1357298332    60282569   
47     1357298322    60282460   
52     1357294630    46500014   
55     1336502906    45485640   
63     1345543925    40482361   
102    1366671502    69769248   
115    1374744563    69754500   
117    1374744058    69754493   
118    1374743979    69754492   
119    1359946464    69754465   
120    1376117980    69755215   
121    1374744537    69754499   
122    1359946299    69758687   
123    1374744774    69754445   
124    1359947372    69754468   
125    1374744669    69755087   
126    1376118698    69755385   
127    1359946243    69758728   
128    1359946218    69758694   
129    1359946213    69758720   
130    1359946209    69759007   
131    1359946028    69758726   
132    1359945993    69758696   
133    1359945944    69758708   
134    1374744092    69754444   
135    1376118137    69755386   
140    1359946199    69758682   
141    1359946145    69758859   
151    1341653211    69747246   
160    1374744310    69755078   
161    1374744276    69755086   
167    1370722816    69735788   
168    1370722814    69735786   
300    1329785535    69736968   
301    1370721470    69727123   
302    1370721458    69727157   
303    1370721454    69727150   
315    1339019225    69646762   
316    1131952101    69682862   
586    1377456208    69591602   
587    1377456179    69591681   
588    1377456135    69591646   
589    1377456126    69591639   
741     988454178    69509145   
936    1357213422    68531980   
937    1357213410    68532901   
938    1357213407    68532826   
939    1357213406    68532819   
950    1357213230    68532680   
951    1357213229    68532109   
952    1357213223    68532895   
953    1357213219    68532796   
972     817443607    68951160   
1088   1357211554    68531744   
1089   1357211552    68531720   
1090   1357211551    68531690   
1091   1357211550    68532512   
1092   1357211549    68532475   
1233   1357210567    68532123   
1234   1357210565    68532857   
1361    817612591    68411534   
1364    817612427    68410483   
1365    994667771    68566975   
1366    994667724    68567071   
1376    817612341    68557744   
1377    817612256    68557904   
1393   1357204343    68397296   
1424   1357201791    68532772   
1426   1357201768    68532543   
1592    817368428    68046590   
1665    817360881    67641437   
1666    817360862    67643158   
1667    817597388    67640096   
1670    817598239    67681679   
1673    817360910    67641680   
1674    817360893    67643622   
1711    817434951    67110698   
1712    988682681    67110452   
1748   1357174918    66784524   
1749    988674767    66784784   
1760   1357172234    66573333   
2061    817324096    65956199   
2119   1329680665    65527450   
2141   1071734102    65332283   
2148    817311069    65314876   
2163    817562038    65307762   
2246   1376061579    69327650   
2247   1376061547    69327506   
2248   1376061538    69327810   
2251   1378783628    60838964   
2286    817554532    64628462   
2587   1170721347    63425161   
2957    817531878    62786911   
2996    817528228    60876522   
3024    817260169    61772250   
3093    817550725    64220352   
3318   1366512159    60187963   
3329   1347829742    47228214   
3571   1374565649    45519871   
3607   1374562913    44330422   
3636   1374560802    43378296   
3637   1374560761    43375462   
3638   1374560

In [10]:
height_new=df[(df['matches_length'].astype(str)=='[]')&(df['matches_height'].astype(str)!='[]')&(df['matches_ft'].astype(str)=='[]')&(df['matches_diameter'].astype(str)=='[]')]#&(df['new_matches'].astype(str)!='[]')]
height_new['matches_height']=height_new['matches_height'].apply(lambda x: natsorted(x))
height_new['matches_height']=height_new['matches_height'].apply(lambda x: re.sub(r'\\" H|"H|H|h',' in',str(x))).apply(lambda x: re.sub(r'\\" H|"H|\"xa0H|xa0H|"','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
height_new['matches_height']=height_new['matches_height'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
# df_new['matches']=df_new['matches'].apply(lambda x: round_string_float(x))
height_new['matches_height']=height_new['matches_height'].apply(lambda x: re.sub('\\\" H',' in',str(x))).apply(lambda x: re.sub('\\\\','',str(x))).apply(lambda x: re.sub(',','","',str(x))).apply(lambda x: re.sub('" | "','"',str(x)))
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A
height_new['matches_height']=height_new['matches_height'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
height_new['matches_height']=height_new['matches_height'].apply(lambda x: str(x)).apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub('" | "','"',str(x))).apply(lambda x: re.sub('  ',' ',str(x))).apply(lambda x: re.sub('\”|\)|inig','',str(x))).apply(lambda x: re.sub('  ',' ',str(x)))
del height_new['matches_width']
del height_new['matches_diameter']

height_new['matches_height']=height_new['matches_height'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
height_new['matches_height']=height_new['matches_height'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
height_new['matches_height']=height_new['matches_height'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))


In [11]:
print(len(height_new))
# height_new[0:500]

155


In [12]:
height_new[curation_col]=height_new['matches_height']
matches=height_new[['external_id',curation_col]]
print(len(matches))
matches.head(100)

155


external_id                   Q:length
17       77417855                  ["42 in"]
29       63253023          ["15 in","30 in"]
50       45823848                  ["30 in"]
147      69749804                  ["54 in"]
303      69727150                  ["30 in"]
1423     68480912                  ["23 in"]
1807     76480119                ["9.75 in"]
2118     65578940                  ["38 in"]
2251     60838964                  ["36 in"]
2587     63425161                  ["42 in"]
2928     61936713                ["29.5 in"]
2981     62388498                  ["38 in"]
3652     44831981               ["30.37 in"]
3915     46745538               ["25.75 in"]
7682     69749195        ["12 in","14.5 in"]
7701     69612573                   ["9 in"]
7808     69768686                  ["18 in"]
9453     62886345                 ["3.5 in"]
10936    64802107       ["28.25 in","35 in"]
10937    68254711               ["48.35 in"]
10943    43484737          ["11 in","48 in"]
11826    61868090  ["39 in","41 in","42 in"]
11848    43668526                ["27.9 in"]
12411    62109963        ["36 in","37.5 in"]
12561    67472796                   ["9 in"]
12873    69694586                ["38.5 in"]
13155    68418038                  ["54 in"]
13401    66302186                ["50.2 in"]
13404    69716415                  ["21 in"]
13405    60552730                  ["46 in"]
13406    64625744                   ["3 in"]
14732    46745521               ["25.75 in"]
14733    46753236       ["40 in","74.75 in"]
14768    69714137                 ["2.1 in"]
15339    69614226                  ["30 in"]
15412    68255688               ["28.25 in"]
15433    66518822                  ["19 in"]
15552    63240238                ["43.5 in"]
15685    42519744                  ["35 in"]
16233    41189368                  ["12 in"]
16793    69345135               ["11.75 in"]
18842    43699421                ["11.8 in"]
22254    60289896               ["41.75 in"]
22375    69735653                  ["24 in"]
23600    61949812     ["29.5 in","43.75 in"]
23601    62109932        ["36 in","37.5 in"]
23997    69718433                  ["16 in"]
23998    69718431                  ["16 in"]
24713    68254919                ["71.9 in"]
24912    67666560                ["40.5 in"]
24932    67547685                  ["64 in"]
25478    64897707               ["40.00 in"]
26235    62679497                   ["9 in"]
30162    69749188                ["15.5 in"]
30189    60263773                  ["40 in"]
30190    63658606         ["7.5 in","18 in"]
30231    17956485            ["3 in","4 in"]
31441    68886967                  ["30 in"]
31809    68886981                  ["30 in"]
32254    69681891                  ["40 in"]
32256    62655545                  ["29 in"]
32259    68418052                  ["54 in"]
32261    65590898          ["17 in","18 in"]
33102    69498373                   ["6 in"]
33240    47468269                ["26.3 in"]
33406    69612553                   ["9 in"]
33427    67213764                  ["19 in"]
33608    69493974               ["11.02 in"]
33611    40974316  ["2 in","6.5 in","36 in"]
33612    68886974                  ["30 in"]
34307    62105712    ["32.72 in","37.56 in"]
34761    69716417                  ["24 in"]
34762    69716421                 ["108 in"]
34968    69716422                 ["108 in"]
35212    65313725                  ["35 in"]
35925    69735664                  ["24 in"]
36134    67565658                  ["29 in"]
41044    68782924                ["39.4 in"]
41048    64802091       ["28.25 in","35 in"]
41741    69768703                   ["9 in"]
42495    41583142               ["15.63 in"]
42742    41190609                ["33.5 in"]
42925    69735672                  ["36 in"]
42942    66546252                  ["46 in"]
42986    18818930                ["13.5 in"]
43642    69587892                ["30.5 in"]
44385    42457800                ["22.9 in"]
44513    64690971                  ["13 in"]
4

In [13]:
# print(len(matches[curation_col].explode().value_counts()))
# matches[curation_col].explode().value_counts()

# Length-Good Stuff

In [14]:
from natsort import natsorted
regex_pattern=r'(?i)((?<!Spacing between.{0,16})(?<!Cord Measures.{0,10})\d+\.\d+[\W]{0,2}"[\W]{0,2}(?!let)(?!leg)Le?n?g?t?h?\b)|((?<!Spacing between.{0,16})(?<!Cord Measures.{0,10})\d+.?\d+\/\d+[\W]{0,2}"[\W]{0,2}(?!let)(?!leg)Le?n?g?t?h?\b)|((?<!Spacing between.{0,16})(?<!Cord Measures.{0,10})(?<!\.)(?<!\/)\d+(?!\.\d)[\W]{0,2}"[\W]{0,2}(?!let)(?!leg)Le?n?g?t?h?\b)|((?<!Spacing between.{0,16})(?<!Cord Measures.{0,10})(?<!\.)(?<!\/)\d+(?!\.)[\W]{0,2}"[\W]{0,2}(?!let)(?!leg)Le?n?g?t?h?\b)|()'           
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df[df['matches'].astype(str)!='[]']))
df['matches']=df['matches'].apply(lambda x: natsorted(x))

691


In [18]:
regex_pattern_blank=r'\d'
df['new_matches'] = df['matches'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df_new=df[(df['new_matches'].astype(str)!='[]')&(df['matches'].astype(str)!='[]')]
df_new['matches']=df_new['matches'].apply(lambda x: re.sub(r'\\" L|"L|L|l',' in',str(x))).apply(lambda x: re.sub(r'\\" L|"L|\"xa0L|xa0L|"','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
df_new.head(5)

product_id external_id  \
22    817187575    62437462   
32    817190020    62784450   
49    817180054    60583932   
102  1366671502    69769248   
103  1376120707    69768707   

                                                                product_name  \
22                      Hudson Baby® Sailor Whale Hooded Towel in Blue/White   
32                         Modway Marina Outdoor Patio Side Table in Natural   
49                                                Bedside Econorail in Black   
102                        Bubbles Personalized 30-Inch x 60-Inch Bath Towel   
103  Ridge Road Décor Aluminum Contemporary Candle Holder in Gold (Set of 2)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                long_desc  \
22                                                                                                                                                                                                                                                                                                                               Make bath time more fun for your little one with the Hudson Baby Sailor Bear Hooded Towel. Alongside the generous size of this towel, made of cozy, absorbent, 100% cotton, this cute cover up's hood features an adorable embroidered sailor whale. Generously sized baby towel Features embroidered sailor whale Blue/white Measures 33\" L x 33\" W 100% cotton Machine wash Imported   
32                                                                                                                                                                                                                           Sit back and enjoy the weather with the Marina Outdoor Patio Side Table. Teak wood construction in a natural finish shows off the wood grain for a look that is simply elegant, making it a perfect addition to your outdoor space. Modern design looks new and welcoming Richly textured wood graining Teak is known to withstand extreme weather conditions Teak wood construction Measures 19.5\" L x 19.5\" W x 19.5\" H 353 lb. weight capacity 1-year manufacturer's warranty Imported   
49   The lightweight Bedside Econorail has a portable design and makes it easier to get in and out of bed safely. Designed to work with regular beds and most hospital beds, it fits beneath your mattress and straps to your bed frame for stability and security. The Bedside Econorail helps you get in and out of bed with ease at home or when traveling Provides reliable assistance into and out of bed Mounts beneath your mattress and straps to the bedframe Lightweight and portable Includes: Bed rail Organizer pouch for storing essentials Carry Case Easy no-tool setup Supports up to 300 lb. Measures 21.5\" L x 20\" W x 16\" D Weighs 1.5 lb. Metal Limited lifetime manufacturer's warranty Imported   
102                                                                                                                                                                                                                                                                                                                            Add a fun and colorful touch to your bathroom with the Bubbles Personalized Bath Towel. The towels will be personalized with any name printed in your choice of color. Personalize with any name Choose from multiple 

In [19]:
# df_new['matches']=df_new['matches'].apply(lambda x: round_string_float(x))
df_new['matches']=df_new['matches'].apply(lambda x: re.sub('\\\" L',' in',str(x))).apply(lambda x: re.sub('\\\\','',str(x))).apply(lambda x: re.sub(',','","',str(x))).apply(lambda x: re.sub('" | "','"',str(x)))
df_new.head(5)

product_id external_id  \
22    817187575    62437462   
32    817190020    62784450   
49    817180054    60583932   
102  1366671502    69769248   
103  1376120707    69768707   

                                                                product_name  \
22                      Hudson Baby® Sailor Whale Hooded Towel in Blue/White   
32                         Modway Marina Outdoor Patio Side Table in Natural   
49                                                Bedside Econorail in Black   
102                        Bubbles Personalized 30-Inch x 60-Inch Bath Towel   
103  Ridge Road Décor Aluminum Contemporary Candle Holder in Gold (Set of 2)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                long_desc  \
22                                                                                                                                                                                                                                                                                                                               Make bath time more fun for your little one with the Hudson Baby Sailor Bear Hooded Towel. Alongside the generous size of this towel, made of cozy, absorbent, 100% cotton, this cute cover up's hood features an adorable embroidered sailor whale. Generously sized baby towel Features embroidered sailor whale Blue/white Measures 33\" L x 33\" W 100% cotton Machine wash Imported   
32                                                                                                                                                                                                                           Sit back and enjoy the weather with the Marina Outdoor Patio Side Table. Teak wood construction in a natural finish shows off the wood grain for a look that is simply elegant, making it a perfect addition to your outdoor space. Modern design looks new and welcoming Richly textured wood graining Teak is known to withstand extreme weather conditions Teak wood construction Measures 19.5\" L x 19.5\" W x 19.5\" H 353 lb. weight capacity 1-year manufacturer's warranty Imported   
49   The lightweight Bedside Econorail has a portable design and makes it easier to get in and out of bed safely. Designed to work with regular beds and most hospital beds, it fits beneath your mattress and straps to your bed frame for stability and security. The Bedside Econorail helps you get in and out of bed with ease at home or when traveling Provides reliable assistance into and out of bed Mounts beneath your mattress and straps to the bedframe Lightweight and portable Includes: Bed rail Organizer pouch for storing essentials Carry Case Easy no-tool setup Supports up to 300 lb. Measures 21.5\" L x 20\" W x 16\" D Weighs 1.5 lb. Metal Limited lifetime manufacturer's warranty Imported   
102                                                                                                                                                                                                                                                                                                                            Add a fun and colorful touch to your bathroom with the Bubbles Personalized Bath Towel. The towels will be personalized with any name printed in your choice of color. Personalize with any name Choose from multiple 

In [20]:
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A
df_new['matches']=df_new['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
df_new.head(5)


SyntaxError: unexpected EOF while parsing (<unknown>, line 1)

In [21]:
df_new['matches']=df_new['matches'].apply(lambda x: str(x)).apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub('" | "','"',str(x))).apply(lambda x: re.sub('  ',' ',str(x))).apply(lambda x: re.sub('\”|en|g|t|h|;|xa0|:','',str(x)))
del df_new['new_matches']
del df_new['matches_height']
del df_new['matches_width']
del df_new['matches_diameter']
del df_new['matches_ft']
df_new.head(5)

product_id external_id  \
22    817187575    62437462   
32    817190020    62784450   
49    817180054    60583932   
102  1366671502    69769248   
103  1376120707    69768707   

                                                                product_name  \
22                      Hudson Baby® Sailor Whale Hooded Towel in Blue/White   
32                         Modway Marina Outdoor Patio Side Table in Natural   
49                                                Bedside Econorail in Black   
102                        Bubbles Personalized 30-Inch x 60-Inch Bath Towel   
103  Ridge Road Décor Aluminum Contemporary Candle Holder in Gold (Set of 2)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                long_desc  \
22                                                                                                                                                                                                                                                                                                                               Make bath time more fun for your little one with the Hudson Baby Sailor Bear Hooded Towel. Alongside the generous size of this towel, made of cozy, absorbent, 100% cotton, this cute cover up's hood features an adorable embroidered sailor whale. Generously sized baby towel Features embroidered sailor whale Blue/white Measures 33\" L x 33\" W 100% cotton Machine wash Imported   
32                                                                                                                                                                                                                           Sit back and enjoy the weather with the Marina Outdoor Patio Side Table. Teak wood construction in a natural finish shows off the wood grain for a look that is simply elegant, making it a perfect addition to your outdoor space. Modern design looks new and welcoming Richly textured wood graining Teak is known to withstand extreme weather conditions Teak wood construction Measures 19.5\" L x 19.5\" W x 19.5\" H 353 lb. weight capacity 1-year manufacturer's warranty Imported   
49   The lightweight Bedside Econorail has a portable design and makes it easier to get in and out of bed safely. Designed to work with regular beds and most hospital beds, it fits beneath your mattress and straps to your bed frame for stability and security. The Bedside Econorail helps you get in and out of bed with ease at home or when traveling Provides reliable assistance into and out of bed Mounts beneath your mattress and straps to the bedframe Lightweight and portable Includes: Bed rail Organizer pouch for storing essentials Carry Case Easy no-tool setup Supports up to 300 lb. Measures 21.5\" L x 20\" W x 16\" D Weighs 1.5 lb. Metal Limited lifetime manufacturer's warranty Imported   
102                                                                                                                                                                                                                                                                                                                            Add a fun and colorful touch to your bathroom with the Bubbles Personalized Bath Towel. The towels will be personalized with any name printed in your choice of color. Personalize with any name Choose from multiple 

In [22]:
df_new['matches']=df_new['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
df_new.head(5)

SyntaxError: unexpected EOF while parsing (<unknown>, line 1)

In [ ]:
df_new['matches']=df_new['matches'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
df_new.head(5)

In [ ]:
df_new['matches']=df_new['matches'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))
df_new.head(5)

In [20]:
df_new[df_new['external_id'].astype(str)=='69760259']

product_id external_id  \
4310  1354545475    69760259   

                                             product_name  \
4310  Max Transitional Square Throw Pillow in White/Black   

                                                                                                                                                                                                                                                                                                                                                                                                                     long_desc  \
4310  Accentuate your bed, sofa, or loveseat with the Max Transitional Square Throw Pillow. Crafted of soft cotton and the perfect layering piece, the pillow adds touch-worthy texture and bohemian charm to your living space. Pillow features touch-worthy texture adding bohemian charm to your living space Square shape Zipper closure Spot clean only 100% cotton 100% polyester fill Measures 20\" W x 20\" L Imported   

      customer_id attributes                   buckets  bucket_id values  \
4310            5       None  Throw/Decorative Pillows       7511   None   

     matches_length blank_matches blank_matches_long    matches  
4310       [20\" L]            []                 []  ["20 in"]

In [21]:
print(len(df_new))
df_new[0:500]

1845


product_id external_id  \
6      1349332160    61515420   
16     1357299512    60748829   
18     1346699985    60730336   
19     1346699984    60730329   
21     1349329642    60483119   
24     1357296921    47362628   
29      817175513    47230361   
46      817162943    42409021   
108    1365301928    69761862   
112    1365300660    69761304   
119    1354545534    69760292   
120    1354545531    69760286   
121    1354545525    69760323   
122    1354545517    69760276   
123    1354545507    69760308   
124    1354545502    69760302   
125    1354545501    69760301   
126    1354545497    69760297   
127    1357273999    69759919   
128    1354545471    69760254   
131    1329790953    69748108   
143    1350635668    69515800   
144    1350635662    69515794   
151    1361306755    69745637   
161    1365292331    69736428   
230    1365287231    69700910   
280    1108562093    69687904   
300    1365277179    69718969   
376    1365275697    69718972   
409    1328467082    69635975   
410    1328467081    69635976   
446    1174819857    69620805   
479    1261233310    69614394   
490     988759294    69589377   
614     817229901    45526367   
634    1349258926    68217853   
669     817402269    69335044   
699    1350581745    69322914   
700    1350581742    69322877   
790    1357211557    68531775   
862     817382892    68755829   
863     817612857    68755805   
952     988711345    68318956   
961     817605559    67858460   
1147   1357173602    66663881   
1148   1357173600    66663836   
1189    817334519    66421047   
1435    988646724    65438480   
1438   1329677176    65163313   
1443   1095512070    64672144   
1444   1349187854    64339160   
1503   1095512073    64672175   
1621   1337598336    63694673   
1638    817287397    63854084   
2072   1349156992    62121880   
2413    817665682    60488008   
2439   1349143689    60310507   
2456   1349142109    67721931   
2460    817243930    47537354   
2522   1349138362    47063129   
2580    817240745    47321830   
2602   1349136291    46643957   
2809    817196095    40386256   
2813   1291498785    69733078   
2814   1361114720    69729553   
2820   1317921575    69686171   
2826   1079389252    69642487   
2830    988565499    69615836   
2831   1058098244    69640291   
2834    988563810    69592270   
2835    988563797    69592281   
2836    988563793    69592283   
2838    817639288    69528171   
2843    988563588    69575643   
2846    817419344    69524462   
2854    817417650    69518652   
2855    817627153    69275890   
2858    817403143    69386596   
2859    817410816    69497011   
2864    817631139    69486157   
2868   1203344992    69689151   
2872    817382697    68796884   
2877    817379521    68652531   
2880   1203159862    68962326   
2889    817609116    68684556   
2899   1171819848    67373185   
2904    817319483    65734049   
2907    817352877    67297153   
2919    817302286    64751009   
2928    817563475    65411230   
2931    817208979    61697409   
2941    817269681    62962728   
2944   1359721153    46198716   
2948    988533513    68521127   
2949    817224004    43921638   
2950    817498496    60065438   
2951    817253981    61192379   
2961    817474339    61766976   
2963    817664042    62090155   
3012   1359695901    69765085   
3030   1365049815    69761194   
3060   1358319114    69736463   
3061   1358319112    69736462   
3062   1358319109    69736459   
3063   1361054363    69736478   
3064   1358319096    69736476   
3065   1358319089    69736470   
3086   1292805055    69720201   
3087   1358313843    69718205   
3123   1365036632    69713821   
3139   1206945871    69708755   
3206   1067904451    69644170   
3247    988491345    69637094   
3248    988491343    69637093   
3330    988480348    69603091   
3353    988477513    69596623   
3372    817642578    69582465   
3411    988465600    69545443   
3449   1364970087    69305474   
3475    817620919    69050824   
3542    817324

In [23]:
df_new[curation_col]=df_new['matches']
new_matches=df_new[['external_id',curation_col]]
print(len(new_matches))
new_matches.head(1)

1845


external_id   Q:length
6    61515420  ["10 in"]

In [25]:
new_matches[0:500]

external_id  \
6        61515420   
16       60748829   
18       60730336   
19       60730329   
21       60483119   
24       47362628   
29       47230361   
46       42409021   
108      69761862   
112      69761304   
119      69760292   
120      69760286   
121      69760323   
122      69760276   
123      69760308   
124      69760302   
125      69760301   
126      69760297   
127      69759919   
128      69760254   
131      69748108   
143      69515800   
144      69515794   
151      69745637   
161      69736428   
230      69700910   
280      69687904   
300      69718969   
376      69718972   
409      69635975   
410      69635976   
446      69620805   
479      69614394   
490      69589377   
614      45526367   
634      68217853   
669      69335044   
699      69322914   
700      69322877   
790      68531775   
862      68755829   
863      68755805   
952      68318956   
961      67858460   
1147     66663881   
1148     66663836   
1189     66421047   
1435     65438480   
1438     65163313   
1443     64672144   
1444     64339160   
1503     64672175   
1621     63694673   
1638     63854084   
2072     62121880   
2413     60488008   
2439     60310507   
2456     67721931   
2460     47537354   
2522     47063129   
2580     47321830   
2602     46643957   
2809     40386256   
2813     69733078   
2814     69729553   
2820     69686171   
2826     69642487   
2830     69615836   
2831     69640291   
2834     69592270   
2835     69592281   
2836     69592283   
2838     69528171   
2843     69575643   
2846     69524462   
2854     69518652   
2855     69275890   
2858     69386596   
2859     69497011   
2864     69486157   
2868     69689151   
2872     68796884   
2877     68652531   
2880     68962326   
2889     68684556   
2899     67373185   
2904     65734049   
2907     67297153   
2919     64751009   
2928     65411230   
2931     61697409   
2941     62962728   
2944     46198716   
2948     68521127   
2949     43921638   
2950     60065438   
2951     61192379   
2961     61766976   
2963     62090155   
3012     69765085   
3030     69761194   
3060     69736463   
3061     69736462   
3062     69736459   
3063     69736478   
3064     69736476   
3065     69736470   
3086     69720201   
3087     69718205   
3123     69713821   
3139     69708755   
3206     69644170   
3247     69637094   
3248     69637093   
3330     69603091   
3353     69596623   
3372     69582465   
3411     69545443   
3449     69305474   
3475     69050824   
3542     65965887   
3597     43468409   
3606     65164396   
4061     47278615   
4076     45845451   
4201     63947021   
4204     69729555   
4205     69736472   
4208     69696843   
4228     69711413   
4288     68726911   
4300     45790430   
4302     44616304   
4304     69745029   
4305     65836279   
4306     69229619   
4309     76360921   
4310     69760259   
4312     46969132   
4325     46734020   
4326     42405092   
4348     69760335   
4349     69760270   
4350     69760309   
4351     69760260   
4357     69515805   
4428     69604829   
4648     68420017   
4695     67795918   
4750     66663874   
4765     66663539   
4928     64416267   
5307     69634068   
5315     69526098   
5319     65877401   
5327     61738669   
5329     61456440   
5330     60053060   
5333     43264407   
5452     69545447   
5472     69516142   
5491     66822714   
5677     46971135   
5767     44328160   
5778     67536986   
5786     69720205   
5790     69760261   
5791     69710278   
5805     12044810   
5831     69768996   
5879     69516192   
5921     69040153   
5970     66545552   
6003     64339177   
6122     69592273   
6123     69592272   
6125     68417604   
6126     69676357   
6129     65992708   
6134     66652441   
6136     65877371   
6142     62962681   
6161     69737103   
6236     69593035   
6252     68066710   
6308     62050128   
6390     69760333   
6391     67461196   
6398     69760319   
6399     6976029

In [24]:
new_matches[curation_col].explode().value_counts()

["20 in"]                                   50
["18 in"]                                   42
["30 in"]                                   39
["24 in"]                                   31
["36 in"]                                   25
                                            ..
["24 in","31.5 in"]                          1
["81.5 in"]                                  1
["39 in","40.25 in"]                         1
["71 in"]                                    1
["13.5 in","15.5 in","31 in","40.25 in"]     1
Name: Q:length, Length: 680, dtype: int64

In [18]:
# print(len(df_new))
# df_new[500:1000]
# print(len(df_new['matches'].explode().value_counts()))
# df_new['matches'].explode().value_counts()[150:650]

# NA Values

In [19]:
regex_pattern_na=r'(?i)((?<!Cord Measures.{0,10})\d+\.\d+.{0,2}".{0,2}[LWHD]?\b)|((?<!Cord Measures.{0,10})\d+.?\d+\/\d+.{0,2}".{0,2}[LWHD]?\b)|((?<!Cord Measures.{0,10})(?<!\.)(?<!\/)\d+(?!\.\d).{0,2}".{0,2}[LWHD]?\b)|((?<!Cord Measures.{0,10})(?<!\.)(?<!\/)\d+(?!\.).{0,2}".{0,2}[LWHD]?\b)|((?<!\.)\d+(?!\.\d).{0,3}?diam)|(measure)|()'
regex_pattern_measure=r'(?i)(measure)|()'
df_na=df
df_na['na_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))
df_na['na_measure_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_measure, x))
print(len(df_na[df_na['na_matches'].astype(str)=='[]']))
df_na=df_na[(df_na['na_matches'].astype(str)=='[]')&(df_na['na_measure_matches'].astype(str)=='[]')]

2332


In [20]:
df_na[curation_col]='n/a'
matches_na=df_na[['external_id',curation_col]]
matches_na

external_id Q:length
8        69715304      n/a
75       69675378      n/a
241      11279090      n/a
253      60282484      n/a
254      60282460      n/a
...           ...      ...
83146    69428852      n/a
83148    68065126      n/a
83161    69513690      n/a
83182    69601920      n/a
83227    18677512      n/a

[2332 rows x 2 columns]

# send to the folder for upload

In [36]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

In [37]:
looks_good('Bed Bath & Beyond', attribute, df, matches)